In [158]:
# Should have thought about this earlier...
# The input from the problem differs from the example in one fundamental detail...
# I discover this solution by accident, being stuck for a day on this problem because the other method I
# implemented was to slow!
# 
# My first attempt at the problem was something alone the lines of cell 3.
# this worked on the example, but somehow didn't work on the input data. I've soon realized that
# the reason was that the example data is ORDERED in a particular way. Every object in the list is
# immediately followed by an object directly orbiting it! The input data for the problem, unfortunately
# doesn't have this nice property. I spent the whole afternoon trying to avoid the problem, 
# cycling over the input data REPEATEDLY and updating a mega-list called 'Umap' (Universal Map). 
# When a cycle finished, then a new one began, still confronting the input data with the mega-list Umap, 
# and adding new indirect links if necessary. For details about this P.I.T.A. of a solution look at
# the jupyter filed with 'Idiot' in their names...
#
# Just before going to dinner, after trying a lot of solutions to optimize the search, and trying the answer
# 4 times, I realized I was missing something crucial. I thought 'maybe I shuld try to order the input data
# and then use my first solution, which should then work on the ordered input...'. To be honest I wasn't
# sure at all, but at least I would try something new. How to order the set? The recursivity option came quite
# quickly at this point. Here it is. Just bare in mind the implementation details depend on the way
# I decided to load the input data (contained in 'raw_list'). In particular from 'AAA)BBB' to simply 'AAABBB'

def recursive_sort(raw_list, sort_list, match):
    
    for item in raw_list:
        if item[0:3] == match:#
            sort_list.append(item)
            recursive_sort(raw_list, sort_list, item[3:6])#3:6
        
    return
# It is so simple it doesn't require any explanation...



# Function that creates the Universal Map, i.e. a catalogue of all objects and their orbits such that:
# - 'AAABBB' means BBB directly orbits AAA
# - 'AAACCCi' means CCC indirectly  orbits AAA
# Input must be sorted in the sense of 'recursive_sort' function.
def create_UniMap(sort_list):
    
    Universal_map = sort_list[:]

    # This is just for printing something during the loop
    count = 1

    # Now the idea to create the Universal Map. I know this works because
    # In a couple AAABBB let's call AAA body and BBB satellite for the sake of explanation.
    # For every direct orbit in the sorted input 
    for item in sort_list:

        # Some printing
        if count%100 == 0:
            print(count, "/", len(sort_list)) 
        count+= 1
    
        # Check all orbits already in Umap
        for link in Universal_map:
            # If the current body is the satellite for some other body in Umap
            if item[0:3] == link[3:6]:
                # Add a new indirect orbit with that body of Umap and the current satellite
                Universal_map.append(link[0:3] + item[3:6] +'i')
    
    # All this magically works because, thanks to the sorted input, when we add a new element to the
    # Universal Map, we can be assured that all its 'parent' orbits (meaning all its indirect
    # orbits which closer to COM) have already been added. So we have no chance of missing any link.
    return Universal_map
    
    
# This could maybe be implemented inside the recursive_sort function, but I like to keep them
# separated for clarity. This function does a very nice thing: for every object in our list (every star or planet)
# it assigns it a 'COM_value', a numeric value that indicates the 'degree' of the object's orbit
# with respect to COM. For COM this value is 0. For an object directly orbiting COM it is 1 and so on.
# Many object can share the same COM_value obviously. 
# Again, the input of this function must be a sorted list of direct orbits, the output of 'recursive_sort'
# function. Not that implementing a similar function for a non-sorted list would be impossible, 
# but it wouldn't certainly be as little lines of code. Here the pre-sorting guarantees that whenever a new
# object is encountered, all the object that it orbits (directly and indirectly) up until COM have
# already been introduced in the classification, so we are not missing anything. 
def create_COM_classification(sorted_orbits, start):
    
    # Starting from COM
    COM_classification = [[start, 0]]
    
    for orbit in sorted_orbits:
            for item in COM_classification:
                if orbit[0:3] == item[0]:
                    COM_classification.append([orbit[3:6], item[1]+1])
    
    
    return COM_classification


# This function, does something similar to the previous one, but between single objects of our catalogue.
# It calculates the number of orbital transfers needed to bring an object ('start') in orbit around
# another object ('end').
def orbit_transfer_counter(sorted_orbits, start, end):
    
    jump_to = start
    # Counter starts from -1 since if 'start' and 'end' are directly orbiting each other (and thus
    # they appear as 'endstart' in sorted_orbits), we have a total of 0 orbital transfers.
    n_transfers = -1
    
    while jump_to != end:
        
        jump_to = [orbit[0:3] for orbit in sorted_orbits if orbit[3:6] == jump_to][0]
        n_transfers += 1
        
    return n_transfers
    

In [147]:
# Loading and formatting the input data. As explained above, from 'AAA)BBB' to just 'AAABBB'.
# Since all astronomical objects have 3 letters names, this causes no confusion.
catalogue = []
sortalogue = []
with open('input.txt', 'r') as infile:
    for line in infile:
        if line != '\n':
            catalogue.append(line.replace('\n', '').replace(')', ''))            

recursive_sort(catalogue, sortalogue, 'COM') 


In [148]:
# Create Universal Map. Takes a few seconds to run, and output has 135690 elements. Uncomment only if necessary!
Umap = create_UniMap(sortalogue)

# By construction, the total number of direct and indirect orbits it's just the number of items of the
# Universal Map.
print(len(Umap))

100 / 1036
200 / 1036
300 / 1036
400 / 1036
500 / 1036
600 / 1036
700 / 1036
800 / 1036
900 / 1036
1000 / 1036
135690


### Part 2: Shortest path to Santa

In [149]:
# Create a COM_value-based classification, with each object and its corresponding COM_value. For example:
# ['COM', 0]
# ['AAA', 1] (AAA directly orbits COM)
# ['BBB', 2] (BBB directly orbits AAA)
# ...
# ...

#COM_classification = create_COM_classification(sortalogue, 'COM')

In [153]:
# We want to find the shortest path between us ('YOU') and Santa ('SAN'). We can do this in 2 steps:
# step 1 -  we find the furthest (from COM) object which both us and Santa are orbiting: if both us and Santa
#           were to climb our way up to COM, this object is the first one at which we could meet (provided one
#           had the patiene to wait for the other, since our distances from that point may be different)
#
# Let's do this first! 

# Convert Umap to set just because
set_Umap = set(Umap)
# Declare the sets of indirect orbits for both us and Santa
YOU_indirect_orbits = set()
SAN_indirect_orbits = set()

# Fill the sets: if we are orbiting an object, add it to our list, and same for good ol' Santa
for link in Umap:
    if link[3:6] == 'YOU' :
        YOU_indirect_orbits.add(link[0:3])
    if link[3:6] == 'SAN':
        SAN_indirect_orbits.add(link[0:3])

# Now find the intersection of the sets (objects which we both are orbiting), and in this new set
# locate the element which has the greatest distance from COM.
# The intersection, back to list since we need an index to initialize from.
common = list(YOU_link.intersection(SAN_link))


biggest_COM = orbit_transfer_counter(sortalogue, common[0], 'COM')

# Simple for loop to find max values
for item in common:
    temp = orbit_transfer_counter(sortalogue, item, 'COM')
    if temp > biggest_COM:
        biggest_COM = temp
        common_orbit = item
        
print(biggest_COM, common_orbit)

63 JZW


In [157]:
# Now, the orbital transfers needed by me and Santa to orbit a common object will be given by the sum

shortest_path = orbit_transfer_counter(sortalogue, 'YOU', 'JZW')+orbit_transfer_counter(sortalogue, 'SAN', 'JZW')

print(shortest_path)

298
